In [52]:
#!pip install web3

     |████████████████████████████████| 1.6 MB 18.0 MB/s eta 0:00:01
  Using cached lru_dict-1.2.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29 kB)
  Using cached eth_abi-4.1.0-py3-none-any.whl (27 kB)
  Using cached hexbytes-0.3.1-py3-none-any.whl (5.9 kB)
  Using cached eth_typing-3.4.0-py3-none-any.whl (6.0 kB)
  Using cached eth_account-0.9.0-py3-none-any.whl (101 kB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
  Using cached aiohttp-3.8.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached websockets-11.0.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (130 kB)
  Using cached eth_hash-0.5.2-py3-none-any.whl (8.6 kB)
  Using cached jsonschema-4.19.0-py3-none-any.whl (83 kB)
Processing /home/jovyan/.cache/pip/wheels/96/b6/57/4c5ff87470c89ceb3ae21e06d4c4008934ad76eff2d579ba95/pyunormalize-15.0.0-py3-none-any.whl
  Using cached eth_u

In [1]:
import pandas as pd
import requests
import datetime
import time
import json
from web3 import Web3 as web3
import numpy as np
from web3 import Web3

In [ ]:
# Initialize a Web3 instance
infura_project_id = 'YOUR-PROJECT-ID'
infura_url = f'https://mainnet.infura.io/v3/{infura_project_id}'

web3 = Web3(Web3.HTTPProvider(infura_url))

In [ ]:
#introduce a caching layer to reduce API calls

from web3.middleware import geth_poa_middleware, simple_cache_middleware
# add the geth_poa_middleware to handle POA chains
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# add the simple_cache_middleware to cache responses
web3.middleware_onion.add(simple_cache_middleware)

In [ ]:
#load top token addresses and block numbers that will be needed later on
top_tokens_addresses = pd.read_csv('../../utils/price-data/top_tokens_addresses.csv') 
block_numbers_df = pd.read_csv('../../utils/block-numbers/block_numbers_2020-01-2023-04.csv')

<h1 style="color: blue; font-size: 36px; font-family: Arial;">Optimism Gateway Escrow</h1>


In [2]:
#contract-address: 0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1
#created at Jun-22-2021 at block 12686786


In [51]:
import pandas as pd
import requests

# Replace API_KEY with your own Ethplorer API key
API_KEY = "YOUR-API-KEY"

# Replace CONTRACT_ADDRESS with the address of the contract you want to query
CONTRACT_ADDRESS = "0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1"

# Construct the API endpoint URL
url = f"https://api.ethplorer.io/getAddressInfo/{CONTRACT_ADDRESS}?apiKey={API_KEY}"

# Send the API request and parse the response as JSON
response = requests.get(url)
data = response.json()
# Get the list of token balances held by the contract
token_balances = data.get("tokens", [])

# Create an empty DataFrame
token_addresses_held_OptimismGateway = pd.DataFrame(columns=["Token Address"])

# Append each token address to a new row in the DataFrame
for token_balance in token_balances:
    token_address = token_balance.get("tokenInfo", {}).get("address")
    token_addresses_held_OptimismGateway = token_addresses_held_OptimismGateway.append({"Token Address": token_address}, ignore_index=True)

# Print the DataFrame
print(token_addresses_held_OptimismGateway)

                                  Token Address
0    0xdac17f958d2ee523a2206206994597c13d831ec7
1    0x2260fac5e5542a773aa44fbcfedf7c193bc2c599
2    0x514910771af9ca656af840dff83e8264ecf986ca
3    0xd291e7a03283640fdc51b121ac401383a46cc623
4    0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
..                                          ...
220  0xbf067cc9b13d11f7d370b825b4d9f695239ffb9b
221  0x245151454c790eb870498e9e5b590145fac1463f
222  0x9b7469e83a09165ec0c42ed6220456afc1fa6a80
223  0x6bd71a44af6d03477eb6f47a92265bb41e8b33f9
224  0x115ec79f1de567ec68b7ae7eda501b406626478e

[225 rows x 1 columns]


## Find out tokens that are held by the escrow that are also within the top300 tokens of coingekcko

In [52]:
# all token_addresses held by the escrow that also appear in the top 300 of CoinGecko
OptimismGateway_top_token_addresses = pd.merge(top_tokens_addresses, token_addresses_held_OptimismGateway, left_on="token_address", right_on="Token Address")
OptimismGateway_top_token_addresses['token_address'] = OptimismGateway_top_token_addresses['token_address'].apply(lambda x: web3.to_checksum_address(x))


In [13]:
OptimismGateway_top_token_addresses

,coingecko_id,token_address,decimals,Token Address
0,tether,0xdAC17F958D2ee523a2206206994597C13D831ec7,6,0xdac17f958d2ee523a2206206994597c13d831ec7
1,usd-coin,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,6,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
2,matic-network,0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0,18,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0
3,dai,0x6B175474E89094C44Da98b954EedeAC495271d0F,18,0x6b175474e89094c44da98b954eedeac495271d0f
4,uniswap,0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984,18,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
5,wrapped-bitcoin,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,8,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599
6,chainlink,0x514910771AF9Ca656af840dff83E8264EcF986CA,18,0x514910771af9ca656af840dff83e8264ecf986ca
7,the-open-network,0x582d872A1B094FC48F5DE31D3B73F2D9bE47def1,9,0x582d872a1b094fc48f5de31d3b73f2d9be47def1
8,lido-dao,0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32,18,0x5a98fcbea516cf06857215779fd812ca3bef1b32
9,true-usd,0x0000000000085d4780B73119b644AE5ecd22b376,18,0x0000000000085d4780b73119b644ae5ecd22b376


## Batch 1

In [ ]:
#new fin try
def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        print(f"Error getting balance for {token_address}: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token addresses DataFrame
token_addresses_df = OptimismGateway_top_token_addresses

token_holding_address = "0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for each token address
contracts = {}
for _, row in token_addresses_df.iterrows():
    token_address = row['Token Address']
    token_address_checksum = web3.to_checksum_address(token_address)
    contract = web3.eth.contract(address=token_address_checksum, abi=abi)
    contracts[token_address] = contract

# Retrieve the token balances for each token address and block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_df.iloc[538:1000].iterrows():
    print(f"Retrieving token balances for {row['Date']} - Block Number:{row['Block Number']} ")
    for _, row2 in token_addresses_df.iterrows(): 
        token_decimals = row2['decimals']
        token_address = row2['Token Address']
        block_number = row['Block Number']
        contract = contracts[token_address]
        token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
        #print(f"On day {row['Date']} token address {token_address} had a balance of {token_balance} ")
        #print(f"Token balance for address {token_address} at Block-Nr. {row['Block Number']} is: {token_balance}")
        time.sleep(0.5)
        balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': row2['coingecko_id'],
            'token_decimals': row2['decimals'],
            'balance': token_balance / 10**token_decimals,
            'has_error': has_error
        })

L1OptimismGateway_token_balances_per_day_part01 = pd.DataFrame(balances)


In [ ]:
#L1OptimismGateway_token_balances_per_day_part01.to_csv('L1OptimismGateway/L1OptimismGateway_token_balances_per_day_part01.csv', index=False)

## Batch 2

In [ ]:

def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        print(f"Error getting balance for {token_address}: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token addresses DataFrame
token_addresses_df = OptimismGateway_top_token_addresses

token_holding_address = "0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for each token address
contracts = {}
for _, row in token_addresses_df.iterrows():
    token_address = row['Token Address']
    token_address_checksum = web3.to_checksum_address(token_address)
    contract = web3.eth.contract(address=token_address_checksum, abi=abi)
    contracts[token_address] = contract

# Retrieve the token balances for each token address and block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_df.iloc[1000:].iterrows():
    print(f"Retrieving token balances for {row['Date']} - Block Number:{row['Block Number']} ")
    for _, row2 in token_addresses_df.iterrows(): 
        token_decimals = row2['decimals']
        token_address = row2['Token Address']
        block_number = row['Block Number']
        contract = contracts[token_address]
        token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
        #print(f"On day {row['Date']} token address {token_address} had a balance of {token_balance} ")
        #print(f"Token balance for address {token_address} at Block-Nr. {row['Block Number']} is: {token_balance}")
        time.sleep(0.5)
        balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': row2['coingecko_id'],
            'token_decimals': row2['decimals'],
            'balance': token_balance / 10**token_decimals,
            'has_error': has_error
        })

L1OptimismGateway_token_balances_per_day_part02 = pd.DataFrame(balances)


In [18]:
L1OptimismGateway_token_balances_per_day_part02.to_csv('L1OptimismGateway/L1OptimismGateway_token_balances_per_day_part02.csv', index=False)

## Batch 3

In [ ]:

# New function to retrieve Ether balance of a smart contract at a specific block number
def get_contract_eth_balance(escrow_checksum, block_number):
    return web3.eth.get_balance(escrow_checksum, block_identifier=block_number)


escrow = "0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1"
escrow_checksum = web3.to_checksum_address(escrow)

# Retrieve the token and Ether balances for each token address and block number, and store the results in a DataFrame
token_address = "ether"
token_name = "ethereum"
token_decimals = 18

balances = []
for i, row in block_numbers_df.iloc[538:].iterrows():

    block_number = row['Block Number']
    eth_balance = get_contract_eth_balance(escrow_checksum, block_number) / 10**token_decimals # Convert Wei to Ether
    print(f"Retrieving balances for {row['Date']} - Block Number:{row['Block Number']} - balance:{eth_balance}  ")    
    balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': token_name,
            'token_decimals': token_decimals,
            'balance': eth_balance,
        })
    time.sleep(0.5)
        
L1OptimismGateway_token_balances_per_day__eth_part03 = pd.DataFrame(balances)


In [8]:
L1OptimismGateway_token_balances_per_day__eth_part03.to_csv('L1OptimismGateway/L1OptimismGateway_token_balances_per_day__eth_part03.csv', index=False)


<h1 style="color: blue; font-size: 36px; font-family: Arial;">  Dai Escrow</h1>


In [ ]:
#contract-address: 0x467194771dAe2967Aef3ECbEDD3Bf9a310C76C65
#created at jul-07-2021 at block 12781427

In [ ]:
def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        print(f"Error getting token balance: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token address
token_address = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
token_address_checksum = web3.to_checksum_address(token_address)

token_holding_address = "0x467194771dAe2967Aef3ECbEDD3Bf9a310C76C65"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for the token address
contract = web3.eth.contract(address=token_address_checksum, abi=abi)

# Retrieve the token balances for each block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_df.iloc[553:].iterrows():
    
    token_decimals = 18  # Assuming 18 decimal places for the DAI token
    block_number = row['Block Number']
    token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
    print(f"Retrieving token balance for {row['Date']} - Block Number:{row['Block Number']} - balance: {token_balance} ")
    time.sleep(0.5)
    balances.append({
        'date': row['Date'],
        'block_number': block_number,
        'token_address': token_address,
        'token_name': 'dai',  
        'token_decimals': token_decimals,
        'balance': token_balance / 10**token_decimals,
        'has_error': has_error
    })

L1DaiEscrow_balances_per_day = pd.DataFrame(balances)


In [1]:
#L1DaiEscrow_balances_per_day.to_csv('L1DaiEscrow/L1DaiEscrow_balances_per_day.csv', index=False)


<h1 style="color: blue; font-size: 36px; font-family: Arial;">  SNX old Escrow</h1>


In [3]:
#contract-address: 0x045e507925d2e05D114534D0810a1abD94aca8d6
#created at Jan-14-2021 at block 11656238

In [ ]:
def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        print(f"Error getting token balance: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token address
token_address = "0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F"
token_address_checksum = web3.to_checksum_address(token_address)

token_holding_address = "0x045e507925d2e05D114534D0810a1abD94aca8d6"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for the token address
contract = web3.eth.contract(address=token_address_checksum, abi=abi)

# Retrieve the token balances for each block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_df.iloc[379:].iterrows():
    
    token_decimals = 18  # Assuming 18 decimal places for the DAI token
    block_number = row['Block Number']
    token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
    print(f"Retrieving token balance for {row['Date']} - Block Number:{row['Block Number']} - balance: {token_balance} ")
    time.sleep(0.5)
    balances.append({
        'date': row['Date'],
        'block_number': block_number,
        'token_address': token_address,
        'token_name': 'snx-synthetix-network-token',  
        'token_decimals': token_decimals,
        'balance': token_balance / 10**token_decimals,
        'has_error': has_error
    })

L1SnxEscrowOld_balances_per_day = pd.DataFrame(balances)


In [16]:
L1SnxEscrowOld_balances_per_day.to_csv('L1SnxEscrowOld/L1SnxEscrowOld_balances_per_day.csv', index=False)


<h1 style="color: blue; font-size: 36px; font-family: Arial;">  SNX current Escrow</h1>

0x5Fd79D46EBA7F351fe49BFF9E87cdeA6c821eF9f
SNX Token

deployed on May-10-2021 

In [ ]:
def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        print(f"Error getting token balance: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token address
token_address = "0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F"
token_address_checksum = web3.to_checksum_address(token_address)

token_holding_address = "0x5Fd79D46EBA7F351fe49BFF9E87cdeA6c821eF9f"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for the token address
contract = web3.eth.contract(address=token_address_checksum, abi=abi)

# Retrieve the token balances for each block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_df.iloc[495:].iterrows():
    
    token_decimals = 18  
    block_number = row['Block Number']
    token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
    print(f"Retrieving token balance for {row['Date']} - Block Number:{row['Block Number']} - balance: {token_balance} ")
    time.sleep(0.5)
    balances.append({
        'date': row['Date'],
        'block_number': block_number,
        'token_address': token_address,
        'token_name': 'snx-synthetix-network-token',  
        'token_decimals': token_decimals,
        'balance': token_balance / 10**token_decimals,
        'has_error': has_error
    })

L1SnxEscrowCurrent_balances_per_day = pd.DataFrame(balances)


In [27]:
L1SnxEscrowCurrent_balances_per_day.to_csv('L1SnxEscrowCurrent/L1SnxEscrowCurrent_balances_per_day.csv', index=False)


<h1 style="color: blue; font-size: 36px; font-family: Arial;">  SNX new Escrow</h1>


In [1]:
#contract-address: 0xCd9D4988C0AE61887B075bA77f08cbFAd2b65068
#created at May-10-2021 at block 12409013

#the escrow is, however, not in use yet

In [ ]:
def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        print(f"Error getting token balance: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token address
token_address = "0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F"
token_address_checksum = web3.to_checksum_address(token_address)

token_holding_address = "0xCd9D4988C0AE61887B075bA77f08cbFAd2b65068"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for the token address
contract = web3.eth.contract(address=token_address_checksum, abi=abi)

# Retrieve the token balances for each block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_df.iloc[495:].iterrows():
    
    token_decimals = 18  
    block_number = row['Block Number']
    token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
    print(f"Retrieving token balance for {row['Date']} - Block Number:{row['Block Number']} - balance: {token_balance} ")
    time.sleep(0.5)
    balances.append({
        'date': row['Date'],
        'block_number': block_number,
        'token_address': token_address,
        'token_name': 'snx-synthetix-network-token',  
        'token_decimals': token_decimals,
        'balance': token_balance / 10**token_decimals,
        'has_error': has_error
    })

L1SnxEscrowNew_balances_per_day = pd.DataFrame(balances)


<h1 style="color: blue; font-size: 36px; font-family: Arial;"> Combining Escrows</h1>


In [16]:
L1OptimismGateway_token_balances_per_day_part01 = pd.read_csv('L1OptimismGateway/L1OptimismGateway_token_balances_per_day_part01.csv') 
L1OptimismGateway_token_balances_per_day_part02 = pd.read_csv('L1OptimismGateway/L1OptimismGateway_token_balances_per_day_part01.csv')
L1OptimismGateway_token_balances_per_day__eth_part03 = pd.read_csv('L1OptimismGateway/L1OptimismGateway_token_balances_per_day__eth_part03.csv')

L1OptimismGateway_token_balances_per_day_part01['escrow_address'] = '0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1'
L1OptimismGateway_token_balances_per_day_part01['escrow_address_rdbl'] = 'L1StandardBridge'
L1OptimismGateway_token_balances_per_day_part01 =L1OptimismGateway_token_balances_per_day_part01.drop(columns=['has_error'])


L1OptimismGateway_token_balances_per_day_part02['escrow_address'] = '0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1'
L1OptimismGateway_token_balances_per_day_part02['escrow_address_rdbl'] = 'L1StandardBridge'
L1OptimismGateway_token_balances_per_day_part02 = L1OptimismGateway_token_balances_per_day_part02.drop(columns=['has_error'])

L1OptimismGateway_token_balances_per_day__eth_part03['escrow_address'] = '0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1'
L1OptimismGateway_token_balances_per_day__eth_part03['escrow_address_rdbl'] = 'L1StandardBridge'


In [17]:
L1DaiEscrow_balances_per_day = pd.read_csv('L1DaiEscrow/L1DaiEscrow_balances_per_day.csv') 

L1DaiEscrow_balances_per_day['escrow_address'] = '0x467194771dAe2967Aef3ECbEDD3Bf9a310C76C65'
L1DaiEscrow_balances_per_day['escrow_address_rdbl'] = 'L1DaiGateway'
L1DaiEscrow_balances_per_day = L1DaiEscrow_balances_per_day.drop(columns=['has_error'])



In [76]:
L1SnxEscrowOld_balances_per_day = pd.read_csv('L1SnxEscrowOld/L1SnxEscrowOld_balances_per_day.csv') 

L1SnxEscrowOld_balances_per_day['escrow_address'] = '0x045e507925d2e05D114534D0810a1abD94aca8d6'
L1SnxEscrowOld_balances_per_day['escrow_address_rdbl'] = 'L1SnxOldBridge'
L1SnxEscrowOld_balances_per_day = L1SnxEscrowOld_balances_per_day.drop(columns=['has_error'])


In [19]:
L1SnxEscrowCurrent_balances_per_day = pd.read_csv('L1SnxEscrowCurrent/L1SnxEscrowCurrent_balances_per_day.csv') 

L1SnxEscrowCurrent_balances_per_day['escrow_address'] = '0x5Fd79D46EBA7F351fe49BFF9E87cdeA6c821eF9f'
L1SnxEscrowCurrent_balances_per_day['escrow_address_rdbl'] = 'L1SnxCurrentdBridge'
L1SnxEscrowCurrent_balances_per_day = L1SnxEscrowCurrent_balances_per_day.drop(columns=['has_error'])


In [20]:
# create a list of the dataframes
dfs = [L1OptimismGateway_token_balances_per_day_part01,
       L1OptimismGateway_token_balances_per_day_part02,
       L1OptimismGateway_token_balances_per_day__eth_part03,
       L1DaiEscrow_balances_per_day, 
       L1SnxEscrowOld_balances_per_day,
       L1SnxEscrowCurrent_balances_per_day]eq

# concatenate the dataframes along the rows
optimism_escrow_balances = pd.concat(dfs, ignore_index=True)

In [ ]:
optimism_escrow_balances['token_address'] = optimism_escrow_balances['token_address'].apply(lambda x: web3.to_checksum_address(x) if x != 'ether' else x)
optimism_escrow_balances['date'] = pd.to_datetime(optimism_escrow_balances['date'])
optimism_escrow_balances = optimism_escrow_balances[optimism_escrow_balances['date'] < '2023-04-30']
optimism_escrow_balances['token_address'] = optimism_escrow_balances['token_address'].apply(str)

In [83]:
price_data = pd.read_csv('../../utils/price-data/price_data.csv') 
price_data['token_address'] = price_data['token_address'].apply(lambda x: web3.to_checksum_address(x) if x != 'ether' else x)
price_data['timestamp'] = pd.to_datetime(price_data['timestamp'])
price_data['token_address'] = price_data['token_address'].apply(str)

In [ ]:
merged_df = optimism_escrow_balances.merge(price_data, left_on=['token_address', 'date'], right_on=['token_address', 'timestamp'], how='left')

In [9]:
optimism_escrow_balances.shape

(28579, 8)

In [13]:
missing_values = merged_df[['timestamp', 'price']].isna().any()
print(missing_values)


timestamp    True
price        True
dtype: bool


there are some rows that could not have merged, due to some missing price data on CoinGecko

In [ ]:
missing_rows = merged_df.loc[merged_df['timestamp'].isna() | merged_df['price'].isna()]
missing_rows


In [16]:
missing_rows["token_address"].value_counts()

0x0C10bF8FcB7Bf5412187A595ab97a3609160b5c6    314
0x4d224452801ACEd8B2F0aebE155379bb5D594381    268
0xF17e65822b568B3903685a7c9F496CF7656Cc6C2    162
0xae78736Cd615f374D3085123A210448E74Fc6393    154
0xC18360217D8F7Ab5e7c516566761Ea12Ce7F9D72    140
0x582d872A1B094FC48F5DE31D3B73F2D9bE47def1     66
Name: token_address, dtype: int64

In [86]:
optimism_escrows_final = optimism_escrow_balances.merge(price_data, left_on=['token_address', 'date'], right_on=['token_address', 'timestamp'])

In [24]:
optimism_escrow_balances.shape

(28579, 8)

In [87]:
optimism_escrows_final.shape

(27475, 10)

In [88]:
optimism_escrows_final['date'] = pd.to_datetime(optimism_escrows_final['date'])
optimism_escrows_final['token_address'] = optimism_escrows_final['token_address'].astype(str)
optimism_escrows_final['timestamp'] = pd.to_datetime(optimism_escrows_final['timestamp'])

In [89]:
optimism_escrows_final['balance_usd'] = optimism_escrows_final['balance'] * optimism_escrows_final['price']


In [91]:
optimism_escrows_final['date'] = pd.to_datetime(optimism_escrows_final['date'])
optimism_escrows_final['token_address'] = optimism_escrows_final['token_address'].astype(str)
optimism_escrows_final['timestamp'] = pd.to_datetime(optimism_escrows_final['timestamp'])

In [92]:
#can be disregarded and will be overwritten with the correct value in eth in the analysis part
optimism_escrows_final['token_amount'] = optimism_escrows_final['balance'] * 10 ** optimism_escrows_final['token_decimals']

def wei_to_eth(wei_value):
    # 1 Ether (ETH) is equivalent to 10^18 Wei
    eth_value = wei_value * 1e-18
    return eth_value

# Apply the conversion function and add a new column
optimism_escrows_final["balance_eth"] = optimism_escrows_final["token_amount"].apply(wei_to_eth)

### save final df

In [12]:
#optimism_escrows_final = pd.read_csv('../../data/optimism_escrows_final.csv')